In [ ]:
# default_exp pipelines

# pipelines

In [ ]:
# export

from simplebitk.utils import dedimensions
from simplebitk.utils import clusters
from simplebitk.utils import plots
from simplebitk import defaults
import pandas as pd
from simplebitk.plots import compare_plots

In [ ]:
# export

def de_dimensions(matrix,prefix,*,sep='\t',dedimensions_method='PCA',cluster_method='MiniBatchKMeans',
                  assess_method='silhouette_score',
                  dimensions=3,
                  cluster_number=None,
                  row_feature=True,annotation=None,size=None,style=None,title=None,fig='png'):
    '''
    
    :param str matrix: matrix table, if row represents feature, please note to add '--row-feature' option
    :param str prefix: output prefix
    :param str sep: separation
    :param str dedimensions_method: de-dimensions method
    :param str cluster_method: cluster method
    :param int dimensions: reduce to n dimensions
    :param int cluster_number: cluster number, if not specific it, it will be the best cluster number infered
    :param bool row_feature: row in the matrix represents feature
    :param str assess_method: assess methods for best cluster number
    :param str annotation: annotation file, sep should be ','
    :param str size: size column in annotation file
    :param str style: style column in annotation file
    :param str title: figure title
    :param str fig: png/pdf
    '''
    df = pd.read_csv(matrix,sep=sep,index_col=0)
    if row_feature:
        df = df.T
    if annotation is not None:
        anno = pd.read_csv(annotation,index_col=0)
    else:
        anno = None
    
    dedim_df,reducer = dedimensions.reduce_dimensional(df,
                                                       method=dedimensions.de_dim_methods[dedimensions_method],
                                                       n_components=dimensions)
    if cluster_number is None:
        best,records,peaks = clusters.find_best_cluster_number(dedim_df,
                                                                  clusters.cluster_methods[cluster_method],
                                                                  defaults.cluster_params,
                                                                  ass_method=clusters.ass_methods[assess_method])
        if len(best) == len(records):
            cluster_number = 2
        else:
            cluster_number = best[0]
    clustering = clusters.clusters_annotation(dedim_df,
                                              clusters.cluster_methods[cluster_method],
                                              defaults.cluster_params)
    dedim_df[getattr(clusters.cluster_methods[cluster_method],'__name__')]= clustering
    dedim_df=pd.concat([dedim_df,anno],axis=1)
    dedim_df.to_csv(prefix+'_'+dedimensions_method+'.csv')
    plots.scatter_plots_for_reduce_dimensional(dedim_df,dedimensions_method+'1',dedimensions_method+'2',
                                               output=prefix+'_'+dedimensions_method+'.'+fig,
                                               hue=getattr(clusters.cluster_methods[cluster_method],'__name__'),
                                               size=size,style=style,
                                               xlabel=dedimensions_method+'1',
                                               ylabel=dedimensions_method+'2',
                                               title=title)

In [ ]:
# export

def scatter_plot(table,output,* ,column1,column2,name1=None,name2=None,logscale=False,value_name='value',sep='\t',\
                                                                                                          hue=None):
    '''
    scatter plot for two columns in the table

    :param str table: table path
    :param str output: output
    :param str column1: the column 1 name
    :param str column2: the column 2 name
    :param str name1: name in the figure for column1
    :param str name2: name in the figure for column2
    :param str value_name: the value name, for example: expression, TPM?
    :param str sep:
    :param str hue:
    :param bool logscale:
    :return:
    '''
    df = pd.read_csv(table,sep=sep,index_col=0)
    if None is name1:
        name1=column1
    if None is name2:
        name2=column2
    compare_plots(df,column1,column2,name1,name2,value_name,output,hue,logscale=logscale)